<a href="https://colab.research.google.com/github/LuanDavid7/AI_EKT/blob/main/AI_Answer_EKT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**CÓDIGO ATUAL**

**Instala Bibliotca faltante**

In [1]:
!pip install python-dotenv

**Código Inicial Final**


Versão com Totos os problemas cadastrados

In [ ]:
import requests
import json
import datetime
import re
import os
import pytz
import time
from dotenv import load_dotenv

# Carregar variáveis de ambiente
load_dotenv()

# Configurações da API
API_KEY = os.getenv("GROQ_API_KEY", "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")
MODEL = "llama3-70b-8192"
URL = "https://api.groq.com/openai/v1/chat/completions"
MAX_RETRIES = 3  # Número máximo de tentativas
INITIAL_DELAY = 1  # Tempo inicial de espera em segundos

# Configurar fuso horário de Brasília
TIMEZONE = pytz.timezone('America/Sao_Paulo')

# Base de conhecimento da Neoenergia Elektro
CATEGORIAS_PROBLEMAS = [
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - EQUIPAMENTO SUPORTE A VIDA", "criticidade": "Emergência", "termos": ["Tenho equipamento de suporte à vida", "Estou com respirador", "Minha máquina de oxigênio parou"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Emergencia médica", "subtipo": "EMERGENCIA MEDICA - HOSPITAL COM PROBLEMA", "criticidade": "Emergência", "termos": ["O hospital está sem luz", "Preciso de energia para um hospital"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia - Geral", "criticidade": "Médio", "termos": ["Minha casa está sem luz", "Só aqui em casa que caiu a energia"], "interno": True},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu relógio digital está apagado", "O medidor novo não está funcionando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Energia Geral - Área Amplia", "criticidade": "Alto", "termos": ["Faltou luz na rua", "A vizinhança inteira sem energia", "Apagão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "FALTA DE FASE", "criticidade": "Médio", "termos": ["Algumas lâmpadas acendem, outras não", "Só um lado da casa tem luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "Falta de Fase em Medidor Eletrônico", "criticidade": "Médio", "termos": ["Meu medidor está piscando em vermelho", "Aparece um erro no painel do relógio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Falta de Energia", "subtipo": "SIGFI FALTA DE ENERGIA FOTOVOLTAICO", "criticidade": "Médio", "termos": ["Meu sistema solar não está gerando energia", "As baterias não estão carregando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "OSCILACÃO DE TENSÃO", "criticidade": "Médio", "termos": ["A luz fica piscando", "ventilador oscilando","Meus aparelhos desligam e ligam sozinhos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO ALTA", "criticidade": "Alto", "termos": ["A luz está muito forte", "Meus aparelhos estão queimando"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Níveis de tensão", "subtipo": "TENSÃO BAIXA", "criticidade": "Médio", "termos": ["A luz está fraca", "Meus aparelhos não funcionam direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Outros", "subtipo": "TRANSFORMADOR", "criticidade": "Alto", "termos": ["O transformador está fazendo barulho", "Tem um cheiro estranho vindo do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "CHOQUE EM INSTALAÇÃO", "criticidade": "Emergência", "termos": ["Estou tomando choque na torneira", "A geladeira está dando choque"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "ELO FUSIVEL QUEIMADO/CAIDO", "criticidade": "Alto", "termos": ["Tem algo pendurado no poste", "Um dispositivo caiu do transformador"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM MEDIDOR", "criticidade": "Médio", "termos": ["Meu relógio de luz está com problema", "O medidor parece quebrado"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM QUICK-LAG", "criticidade": "Médio", "termos": ["O disjuntor do poste parece queimado", "Algo explodiu na caixa do poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FALHA EM RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está solto", "A conexão com o poste está com problema"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO COM ARVORE NA REDE", "criticidade": "Emergência", "termos": ["Está saindo faísca do fio com a árvore", "A árvore está pegando fogo no fio"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["Está saindo fogo dos fios de alta tensão", "Os cabos grossos estão soltando faíscas"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NA REDE SECUNDARIA", "criticidade": "Alto", "termos": ["O fio da rua está soltando faísca", "Está saindo fogo do cabo de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FIO DANDO FOGO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que entra na minha casa está pegando fogo", "Tem faísca na entrada de luz"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO EQUIPAMENTO", "criticidade": "Alto", "termos": ["Roubaram algo do poste", "Levaram equipamento da rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO FIACÃO/PADRÃO", "criticidade": "Alto", "termos": ["Roubaram os fios da minha casa", "Levaram o medidor"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "FURTO REDE", "criticidade": "Alto", "termos": ["Roubaram os fios da rua", "Tem cabos cortados no poste"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "INVERSÃO DE FASE", "criticidade": "Médio", "termos": ["Meu motor está girando ao contrário", "O equipamento trifásico não funciona direito"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE PRIMARIA", "criticidade": "Alto", "termos": ["Tem algo preso nos fios de alta tensão", "Um objeto está encostado na rede"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "OBJETO ESTRANHO NA REDE SECUNDARIA", "criticidade": "Médio", "termos": ["Jogaram algo nos fios", "Tem um objeto preso na fiação"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Provável defeito falha", "subtipo": "POSTE PADRÃO QUEBRADO", "criticidade": "Alto", "termos": ["Meu poste padrão está quebrado", "A entrada de luz está torta"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE PRIMARIA", "criticidade": "Emergência", "termos": ["O fio grosso da alta tensão está partido", "Caiu um cabo de alta tensão"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NA REDE SECUNDARIA", "criticidade": "Emergência", "termos": ["O fio da luz caiu", "Tem um cabo quebrado na rua"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "FIO PARTIDO NO RAMAL DE SERVICO", "criticidade": "Alto", "termos": ["O fio que vem para minha casa está quebrado", "Caiu o cabo da entrada"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO COM FIO PARTIDO", "criticidade": "Emergência", "termos": ["Poste Caiu e fio caiu","Alguém bateu no poste e o fio caiu", "Poste quebrado com fios soltos"], "interno": False},
    {"categoria": "Ocorrência Técnica", "tipo": "Risco de Morte", "subtipo": "POSTE ABALROADO SEM FIO PARTIDO", "criticidade": "Alto", "termos": ["Poste inclinado","O poste está torto depois do acidente", "Bateram no poste mas os fios estão intactos"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Ligação", "subtipo": "Ligação Nova", "criticidade": "Médio", "termos": ["Preciso de uma nova ligação de energia", "Minha casa ainda não tem luz"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação Normal", "criticidade": "Médio", "termos": ["Paguei a conta e a luz não voltou", "Quero religar a energia"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Religação", "subtipo": "Religação de Urgência", "criticidade": "Alto", "termos": ["Preciso da energia hoje mesmo", "Tenho uma pessoa doente em casa"], "interno": False},
    {"categoria": "Ocorrência Comercial", "tipo": "Desligamento progamado", "subtipo": "Desligamento para Manutenção", "criticidade": "Médio", "termos": ["Vai faltar luz aqui?", "Tem alguma manutenção programada?"], "interno": False}
]

def obter_saudacao():
    """Retorna uma saudação baseada no horário de Brasília."""
    try:
        hora_atual = datetime.datetime.now(TIMEZONE).hour
        if 5 <= hora_atual < 12:
            return "Bom dia"
        elif 12 <= hora_atual < 18:
            return "Boa tarde"
        else:
            return "Boa noite"
    except Exception:
        return "Olá"

def mensagem_fechamento():
    """Retorna a mensagem padrão de fechamento."""
    return """Agradecemos seu contato! Sempre que precisar estamos disponíveis nos nossos canais:

💻 Site: https://www.neoenergia.com/web/sp
📱 Aplicativo: Android → https://bit.ly/NELK-Android | IOS → https://bit.ly/IOS-NELK
📞 Teleatendimento: 116
Deficiente auditivo ou de fala | 0800 701 01 55
Atendimento fora da área de concessão | 0800 276 0116
Conte sempre com a gente!"""

def gerar_protocolo():
    """Gera um número de protocolo baseado na data e hora atuais em Brasília."""
    agora = datetime.datetime.now(TIMEZONE)
    return f"{agora.year}{agora.month:02d}{agora.day:02d}{agora.hour:02d}{agora.minute:02d}{agora.second:02d}"

def classificar_problema(mensagem, contexto_adicional=""):
    """Classifica o problema com base na mensagem do cliente e contexto adicional."""
    mensagem = mensagem.lower()
    contexto = contexto_adicional.lower() if contexto_adicional else ""

    # Verificar emergências primeiro (mantido igual)
    for problema in CATEGORIAS_PROBLEMAS:
        if problema.get("criticidade") == "Emergência":
            for termo in problema.get("termos", []):
                if termo.lower() in mensagem or termo.lower() in contexto:
                    return problema

    # Caso específico: medidor apagado + disjuntores ok = problema externo
    if ("medidor" in mensagem or "medidor" in contexto) and ("apagado" in mensagem or "apagado" in contexto):
        if ("disjuntor" in contexto or "disjuntores" in contexto) and ("ligado" in contexto or "ok" in contexto):
            return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia - Geral"), None)

    # Verificar se é falta de energia geral em área ampla
    if ("sem energia" in mensagem or "sem luz" in mensagem) and ("vizinh" in mensagem or "rua" in mensagem or "bairro" in mensagem):
        return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia Geral - Área Amplia"), None)

    # Outros casos específicos (mantido igual)
    for problema in CATEGORIAS_PROBLEMAS:
        for termo in problema.get("termos", []):
            if termo.lower() in mensagem or termo.lower() in contexto:
                return problema

    # Default para falta de energia geral (quando medidor apagado e disjuntores ok)
    if "sem energia" in mensagem or "sem luz" in mensagem:
        return next((p for p in CATEGORIAS_PROBLEMAS if p["subtipo"] == "Falta de Energia - Geral"), None)

    return None

def criar_contexto_ia(historico=[]):
    """Cria um contexto para a IA com informações sobre como deve atuar."""
    return {
        "role": "system",
        "content": """
Você é o assistente virtual da Neoenergia Elektro. Seu objetivo é identificar problemas de energia elétrica e decidir se deve abrir um chamado técnico.

DIRETRIZES IMPORTANTES:
1. Analise a mensagem do cliente para identificar o tipo de problema.
2. Determine se o problema é interno (dentro da casa do cliente) ou externo (responsabilidade da Neoenergia).
3. Para problemas de falta de energia internos, primeiro oriente a verificar disjuntores do quadro interno e padrão de energia.
4. Se o cliente relatar casa toda sem energia, pergunte se há LED aceso no medidor.
5. Para problemas internos, oriente a procurar um eletricista particular.
6. Para problemas que são responsabilidade da Neoenergia, confirme se o cliente deseja abrir um chamado técnico.
7. Só gere protocolo APÓS a confirmação do cliente.
8. Em casos de emergência (fios partidos, choques, etc), comece sua resposta com "ATENÇÃO!" e dê instruções de segurança.
9. Seja objetivo, claro e direto - respostas curtas são preferíveis.
10. Se o cliente começar já descrevendo uma causa de problema, busque identificar de que forma isso afeta ele.
11. Faça no máximo duas perguntas por vez.
12. Se o cliente apontar que um defeito externo está provocando a falha na sua casa, não peça que o cliente faça verificações internas.
13. Se o cliente dizer que não fez as verificações nos disjuntores solicitadas ou se negar a fazer alguma verificação nos disjuntores, o chamado não pode ser gerado, pois pode se tratar de uma falha interna e é recomendado procurar um eletricista particular.
14. Se o disjuntor está desarmando sozinho, o problema é interno.
15. Se o cliente diz estar totalmente sem energia, mas o medidor apresenta sinais de funcionamento (LEDs acesos, display exibindo valores), o problema também é interno.
16. Se o cliente diz estar totalmente sem energia e está com todos os disjuntores ligados e o display apresenta sinal de funcionamento normal, o problema é interno e deve ser recomendado procurar um eletricista particular, pois pode haver algum disjuntor com problema que precise de troca.
17. Em caso de furto oriente o cliente primeiramente a corrigir os problemas gerados na instalaçção interna, para que após isso a equipe atue na parte externa

IMPORTANTE: Se o cliente confirmar que deseja abrir um chamado técnico após apontar indícios de uma falha externa, gere o protocolo IMEDIATAMENTE e não faça mais perguntas sobre o problema.

FLUXO DE ATENDIMENTO:
1. Cumprimento inicial.
2. Identificação do problema.
3. Verificação se é interno ou externo.
4. Para casos internos: orientação (disjuntores, eletricista).
5. Para casos externos: confirmação de abertura de chamado técnico.
6. Geração de protocolo (apenas se confirmado).

ORDEM DOS PROCEDIMENTOS:
1. A saudação é livre e muito simpática.
2. Aguarde o cliente apresentar o problema.
3. Questione e dê as orientações.
4. Analise qual é o problema real.
5. Apenas após isso aponte ao cliente qual é a possível falha.
6. Se ofereça para gerar o chamado assim: "Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Você quer que eu abra um chamado técnico?"
7. SE O CLIENTE CONFIRMAR, GERE O PROTOCOLO. Com base nos seus apontamentos, foi gerado o chamado de número 2025-00001 para solucionar o problema de {problema['tipo']} - {problema['subtipo']}. Não é necessário dizer que o problema é externo, isso é óbvio.
8. Não faça mais perguntas, aguarde o cliente esboçar gratidão e uma despedida, então despeça-se, mas claro não cobre agradecimentos, apenas espere.

Para problemas de emergência, priorize a segurança do cliente antes de qualquer procedimento.

Árvore de Decisão para Identificação de Problemas Elétricos:
1. O cliente está sem energia?
   - Sim → Avance para 2.
   - Não → Avance para 9.

2. Verificação Inicial (Falta de Energia):
   - Pergunta: "Você já verificou os disjuntores do quadro interno e do padrão de entrada? Eles estão ligados?"
     - Sim (disjuntores ligados) → Avance para 3.
     - Não (disjuntores desligados) → Oriente o cliente a religá-los e teste novamente. Se persistir, avance para 3.
   - Pergunta: "O medidor de energia (relógio) está com o display aceso ou LEDs piscando?"
     - Sim (medidor energizado) → Problema interno (verifique instalações do cliente).
     - Não (medidor apagado) → Avance para 3.

3. Escopo da Falta de Energia:
   - Pergunta: "Seus vizinhos também estão sem energia?"
     - Sim (área ampla afetada) → Falta de Energia Geral (relate à equipe de rede).
     - Não (apenas o cliente) → Avance para 4.

4. Tipo de Instalação:
   - Pergunta: "Você possui equipamento de suporte à vida (respirador, oxigênio) afetado?"
     - Sim → Equipamento Suporte a Vida (urgência máxima).
     - Não → Avance para 5.
   - Pergunta: "O problema é em um hospital ou unidade de saúde?"
     - Sim → Hospital com Problema (urgência alta).
     - Não → Avance para 5.

5. Medidor Eletrônico:
   - Pergunta: "O medidor é eletrônico (digital)? Está mostrando alguma mensagem de erro ou luz vermelha?"
     - Sim → Falta de Fase em Medidor Eletrônico ou Falha em Medidor.
     - Não → Avance para 6.

6. Sistema Fotovoltaico:
   - Pergunta: "Você tem sistema de energia solar? Ele está funcionando?"
     - Sim (não está gerando) → SIGFI Falta de Energia Fotovoltaico.
     - Não → Avance para 7.

7. Verificação de Fases:
   - Pergunta: "Algumas tomadas ou lâmpadas funcionam, enquanto outras não?"
     - Sim → Falta de Fase.
     - Não (totalmente sem energia) → Falta de Energia em Ramal de Serviço ou Fio Partido.

8. Danos Visíveis:
   - Pergunta: "Você vê fios partidos, faíscas ou danos no poste/padrão?"
     - Sim → Fio Partido na Rede Secundária/Primária ou Poste Abalroado. Se houver fogo: Fio Dando Fogo.
     - Não → Relate como Falta de Energia Geral (investigação externa).

9. Problemas com Tensão (cliente COM energia):
   - Pergunta: "Você está enfrentando oscilação, tensão alta ou baixa?"
     - Oscilação → "Luzes piscam ou aparelhos reiniciam?" → Oscilação de Tensão.
     - Tensão Alta → "Lâmpadas queimam frequentemente ou brilham muito?" → Tensão Alta.
     - Tensão Baixa → "Lâmpadas fracas ou aparelhos não ligam?" → Tensão Baixa.
   - Perguntas Adicionais para Tensão:
     - "Instalou algum equipamento novo recentemente?"
     - "Há fábricas ou obras próximas?"

10. Problemas no Transformador ou Rede:
    - Pergunta: "Você observa barulhos, cheiros ou vazamentos no transformador?"
      - Sim → Problema no Transformador.
      - Não → Avance para 11.

11. Choques ou Furtos:
    - Pergunta: "Você está levando choque em aparelhos ou torneiras?"
      - Sim → Choque em Instalação.
      - Não → Avance para 12.
    - Pergunta: "Houve furto de cabos, medidor ou equipamentos?"
      - Sim → Furto de Rede/Fiação.

12. Outros Problemas:
    - Pergunta: "Você vê objetos estranhos na rede (pipas, galhos, fios soltos)?"
      - Sim → Objeto Estranho na Rede.
      - Não → Outros (avaliação manual).

Fluxo para Religação/Desligamento:
- Pergunta: "A energia foi cortada por falta de pagamento ou manutenção?"
  - Sim → Verifique comprovante de pagamento e solicite Religação Normal/Urgência.
  - Não → Avance para outras verificações.

Regras-Chave:
- Medidor apagado + disjuntores ligados = Problema externo.
- Medidor aceso + sem energia = Problema interno.
- Tensão irregular = Foco em equipamentos novos ou mudanças no entorno.
- Fogo/fiação exposta = Urgência máxima (risco de incêndio).
"""
    }

def detectar_confirmacao(mensagem):
    """Detecta se a mensagem do cliente é uma confirmação para abrir chamado."""
    confirmacoes = ["sim", "quero", "pode abrir", "abra", "preciso", "com certeza", "abre", "gera", "gere"]
    return any(termo in mensagem.lower() for termo in confirmacoes)

def detectar_negacao(mensagem):
    """Detecta se a mensagem do cliente é uma negação para abrir chamado."""
    negacoes = ["não", "nao", "não precisa", "nao precisa", "não quero", "nao quero", "cancela", "desisto"]
    return any(termo in mensagem.lower() for termo in negacoes)

def detectar_despedida(mensagem):
    """Detecta se a mensagem do cliente é uma despedida."""
    despedidas = ["sair", "tchau", "encerrar", "obrigado", "agradeço", "até mais", "ate mais", "adeus"]
    return any(termo in mensagem.lower() for termo in despedidas)

def consultar_ia(mensagem_cliente, historico=[], problema_identificado=None):
    """Consulta a IA para obter resposta ao cliente."""
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

    messages = [criar_contexto_ia(historico)]

    # Adicionar histórico da conversa
    if historico:
        messages.extend(historico)

    # Adicionar mensagem atual do cliente
    messages.append({"role": "user", "content": mensagem_cliente})

    data = {
        "model": MODEL,
        "messages": messages,
        "temperature": 0.7,
        "max_tokens": 500
    }

    # Implementação de retry com backoff exponencial
    delay = INITIAL_DELAY
    for attempt in range(MAX_RETRIES):
        try:
            response = requests.post(URL, headers=headers, json=data, timeout=10)

            if response.status_code == 200:
                return response.json()["choices"][0]["message"]["content"]

            elif response.status_code == 429:
                error_data = response.json()
                if "try again in" in error_data.get("error", {}).get("message", ""):
                    wait_time = float(error_data["error"]["message"].split("try again in ")[1].split("s")[0])
                    time.sleep(wait_time + 1)
                    continue

            time.sleep(delay)
            delay *= 2

        except Exception as e:
            print(f"Erro ao consultar IA (tentativa {attempt + 1}): {str(e)}")
            if attempt < MAX_RETRIES - 1:
                time.sleep(delay)
                delay *= 2

    # Fallback inteligente baseado no contexto
    contexto = " ".join([msg["content"] for msg in historico if msg["role"] == "user"])
    problema = classificar_problema(mensagem_cliente, contexto)

    if problema:
        if problema["criticidade"] == "Emergência":
            return f"ATENÇÃO! Identificamos um problema de {problema['tipo']} - {problema['subtipo']}. Por segurança, mantenha distância. Posso abrir um chamado de emergência?"
        else:
            return f"Isso parece ser um problema de {problema['tipo']} - {problema['subtipo']}. Deseja que eu abra um chamado técnico?"

    return "Poderia detalhar melhor o problema que está enfrentando?"

def atendimento_cliente():
    """Função principal para iniciar atendimento."""
    print("\n=== Atendimento Neoenergia Elektro ===\n")

    historico = []
    primeira_mensagem = True
    problema_identificado = None
    protocolo_gerado = False

    while True:
        if primeira_mensagem:
            saudacao = f"{obter_saudacao()}! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?"
            print(f"[Atendente] {saudacao}")
            primeira_mensagem = False
            historico.append({"role": "assistant", "content": saudacao})

        mensagem_cliente = input("\n[Cliente] ").strip()

        if detectar_despedida(mensagem_cliente) and not protocolo_gerado:
            print(f"\n[Atendente] {mensagem_fechamento()}")
            break

        historico.append({"role": "user", "content": mensagem_cliente})

        # Se já temos um problema identificado e o cliente confirmou
        if problema_identificado and detectar_confirmacao(mensagem_cliente) and not protocolo_gerado:
            protocolo = gerar_protocolo()
            resposta = f"Chamado técnico aberto para {problema_identificado['tipo']} - {problema_identificado['subtipo']}. Protocolo: {protocolo}. Nossa equipe será acionada."
            print(f"\n[Atendente] {resposta}")
            historico.append({"role": "assistant", "content": resposta})
            protocolo_gerado = True
            continue

        # Consultar a IA
        resposta = consultar_ia(mensagem_cliente, historico, problema_identificado)

        # Atualizar problema identificado se necessário
        if "problema de" in resposta.lower() and not problema_identificado:
            problema_identificado = classificar_problema(mensagem_cliente, resposta)

        print(f"\n[Atendente] {resposta}")
        historico.append({"role": "assistant", "content": resposta})

if __name__ == "__main__":
    atendimento_cliente()


=== Atendimento Neoenergia Elektro ===

[Atendente] Boa tarde! Sou o Elielzinho da Neoenergia Elektro. Como posso ajudar?
